# WeatherPy
----

#### Observations
* Increases in absolute value of latitude is correlated with decreases in temperature.
* Wind speed is weakly correlated with positive increase in latitude. 
* Latitude effects humidity more in the southern than northen hemisphere. 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy


# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

In [ ]:
cities

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
url = "https://api.openweathermap.org/data/2.5/weather?"
units = "metric"
final_url = f'{url}appid={weather_api_key}&units={units}&q='

In [ ]:
weather = requests.get(final_url+"london").json()
weather

url = "https://api.openweathermap.org/data/2.5/weather?"
units = "metric"
final_url = f'{url}appid={weather_api_key}&units={units}&q='

city_list = []
datacounter = 0
for city in cities:
    weather = requests.get(final_url+city).json()
    try:

        weather_data = {
            "City":weather["name"],
            "Lat":weather["coord"]["lat"],
            "Lng":weather["coord"]["lon"],
            "Max Temp":weather["main"]["temp_max"],
            "Humidity":weather["main"]["humidity"],
            "Cloudiness":weather["clouds"]["all"],
            "Wind Speed":weather["wind"]["speed"],
            "Country":weather["sys"]["country"],
            "Date":weather["dt"]    
        }
        city_list.append(weather_data)
        datacounter = datacounter + 1
        print (f'city_list:{weather["name"]},id={weather["id"]}')
    except:
        
        print("no data found:"+city)

In [ ]:
city_list = [] 
datacounter = 0 
for city in cities:
    weather = requests.get(final_url+city).json() 
try:
    weather_data = {
        "City":weather["name"],
        "Lat":weather["coord"]["lat"],
        "Lng":weather["coord"]["lon"],
        "Max Temp":weather["main"]["temp_max"],
        "Humidity":weather["main"]["humidity"],
        "Cloudiness":weather["clouds"]["all"],
        "Wind Speed":weather["wind"]["speed"],
        "Country":weather["sys"]["country"],
        "Date":weather["dt"]    
    }
    city_list.append(weather_data)
    datacounter = datacounter + 1
    print (f'city_list:{weather["name"]},id={weather["id"]}')
except:
    print("no data found:"+city)

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
cities_df = pd.DataFrame(city_list)

In [ ]:
cities_df.to_csv("cities.csv")

In [ ]:
cities_df.describe()

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
humidity = cities_df.loc[cities_df["Humidity"]>100]
humidity
# no cities with humidity >100%

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(cities_df["Lat"], cities_df["Max Temp"])
plt.title("City Latitude vs. Max Temperature")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature(C)");

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(cities_df["Lat"], cities_df["Humidity"])
plt.title("City Latitude vs. Humidity")
plt.xlabel("Latitude")
plt.ylabel("Humidity")

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(cities_df["Lat"], cities_df["Cloudiness"])
plt.title("City Latitude vs. Cloudiness")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)");

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(cities_df["Lat"], cities_df["Wind Speed"])
plt.title("City Latitude vs. Wind Speed")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")

## Linear Regression

In [ ]:
from scipy.stats import linregress

def regress_func(x_values,y_values):  
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
    regress_values = x_values * slope + intercept
    print (rvalue**2)
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
#     plt.annotate(line_eq,(6,10),fontsize=15,color="red")
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r-")
    return line_eq

In [ ]:
north_df = cities_df.loc[cities_df['Lat'] >= 0]
south_df = cities_df.loc[cities_df['Lat'] < 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
max_temp = regress_func(north_df['Lat'], north_df['Max Temp'])
plt.annotate(max_temp,(2,-20),fontsize=15,color="red")
plt.show()

As latitude increases, max temperature decreases. It is an inverse proportion. 

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
max_temp = regress_func(south_df['Lat'], south_df['Max Temp'])
plt.annotate(max_temp,(2,-20),fontsize=15,color="red")
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
humidity = regress_func(north_df['Lat'], north_df['Humidity'])
plt.annotate(humidity,(2,-20),fontsize=15,color="red")
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
humidity = regress_func(south_df['Lat'], south_df['Humidity'])
plt.annotate(humidity,(2,-20),fontsize=15,color="red")
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
cloudiness = regress_func(north_df['Lat'], north_df['Cloudiness'])
plt.annotate(humidity,(2,-20),fontsize=15,color="red")
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
cloudiness= regress_func(south_df['Lat'], south_df['Cloudiness'])
plt.annotate(cloudiness,(2,-20),fontsize=15,color="red")
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
wind_speed = regress_func(north_df['Lat'], north_df['Wind Speed'])
plt.annotate(wind_speed,(2,-20),fontsize=15,color="red")
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
wind_speed = regress_func(south_df['Lat'], south_df['Wind Speed'])
plt.annotate(wind_speed,(2,-20),fontsize=15,color="red")
plt.show()